In [1]:
import cv2
import numpy as np
import face_recognition
from deepface import DeepFace
import joblib

class RealTimeFaceRecognizer:
    def __init__(self, model_path):
        self.model = joblib.load(model_path)

    def recognize_faces_and_emotions(self, video_capture):
        while True:
            # Capture frame-by-frame
            ret, frame = video_capture.read()

            if not ret:
                break

            # Convert the frame from BGR color to RGB color (required for face_recognition library)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Find all the faces and face encodings in the current frame
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            # If faces are detected
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # Perform face recognition
                predictions = self.model.predict([face_encoding])
                name = predictions[0]

                # Draw a rectangle around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

                # Analyze emotion for the face
                try:
                    # Extract the face region as a NumPy array
                    face_np = frame[top:bottom, left:right]

                    emotion_results = DeepFace.analyze(face_np, actions=['emotion'], enforce_detection=False)

                    # Get the dominant emotion
                    emotion = emotion_results[0]['dominant_emotion']

                    # Display emotion label on the rectangle
                    cv2.putText(frame, f"{name} - {emotion}", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

                except ValueError as e:
                    print(e)

            # Display the resulting frame
            cv2.imshow('Real-Time Recognition with Emotion Detection', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release the video capture object and close all windows
        video_capture.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    model_path = "students_trained_model.pkl"  # Path to your trained model
    recognizer = RealTimeFaceRecognizer(model_path)

    # Open the camera
    video_capture = cv2.VideoCapture(0)  # 0 for default camera, change it if you have multiple cameras

    # Perform real-time face recognition with emotion detection
    recognizer.recognize_faces_and_emotions(video_capture)


C:\Users\KCS\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
